In [2]:
import numpy as np
import pandas as pd

In [20]:
# 연평균 상승률의 대푯값 구하기

In [16]:
prod_2 = 100000
prod_1 = 150000
prod = 250000

rt_2 = prod_1 / prod_2
rt_1 = prod / prod_1

rt_mid = ((rt_2 * rt_1) ** (1/2)) * 100

In [18]:
print(rt_mid, '%')

158.11388300841898 %


In [58]:
# 정규분포의 모평균 구간추정: 모분산 모르는 경우. 답지는 표준편차로 했네...

In [44]:
from scipy import stats

n=12
df = 11
mean = 15.5
var = 3.2

rv = stats.t(df)
lcl = mean - rv.isf(0.05) * np.sqrt(var/n)
ucl = mean - rv.isf(0.95) * np.sqrt(var/n)

print(lcl, ucl)

14.572609067393861 16.427390932606137


In [46]:
stats.t.interval(loc=15.5, scale=np.sqrt(var/n), df=df, confidence=0.90)

(14.572609067393861, 16.427390932606137)

In [52]:
# 2 표본 문제에 대한 가설검정. 대응표본, 정규분포 가정 --> 대응비교 t 검정

In [48]:
df = pd.read_csv('27_problem7.csv')

In [50]:
df

,river,up,down
0,a,57.1,45.7
1,b,51.6,37.2
2,c,53.9,53.3
3,d,59.0,54.3
4,e,57.5,46.3


귀무가설_H0 : 상류와 하류의 생물다양성 점수에 차이가 없다.
대립가설_H1 : 상류와 하류의 생물다양성 점수에 차이가 있다.

In [54]:
t, p = stats.ttest_rel(df['up'], df['down'])

print(t,p)

3.3526056764717995 0.028499777234053288


In [56]:
# p-value 0.028로 0.1 미만으로 귀무가설 기각, 대립가설 채택 --> 상류와 하류의 생물다양성 점수에 유의미한 차이가 있다.

Quantile regression 

In [62]:
df_5 = pd.read_csv('27_problem8.csv')

In [66]:
df_5

,temperature,wind,precipitation,user_counts
0,10.400000,4.600000,0.844944,6368
1,5.666667,4.625000,0.040860,5902
2,4.933333,4.725000,0.008696,6226
3,3.400000,2.675000,0.156989,5829
4,8.900000,3.950000,7.988462,7589
...,...,...,...,...
2092,3.500000,2.100000,0.024096,5430
2093,8.233333,2.000000,16.843820,2125
2094,9.500000,3.666667,11.550575,6008
2095,4.800000,1.200000,0.104651,4929


In [70]:
import matplotlib.pyplot as plt
from sklearn.linear_model import QuantileRegressor
from sklearn.model_selection import train_test_split

In [88]:
pd.set_option('display.float_format', lambda x: f'{x:.40f}')

In [168]:
X = df_5[['temperature', 'wind', 'precipitation']]
y = df_5['user_counts']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

model = QuantileRegressor()
model.fit(X_train, y_train)

np.set_printoptions(precision=50)
coef = model.coef_

#####왜 회귀계수 2와 3이 죽었지?????????

print("회귀 계수:", coef)

for idx, coef in enumerate(model.coef_):
    print(f"계수 {idx}: {coef:.50f}")

회귀 계수: [217.19322990126943   0.                 0.              ]
계수 0: 217.19322990126943295763339847326278686523437500000000
계수 1: 0.00000000000000000000000000000000000000000000000000
계수 2: 0.00000000000000000000000000000000000000000000000000


In [170]:
10.5 * model.coef_[0]

2280.528913963329

** type 3 ANOVA

In [176]:
df_55 = pd.read_csv('27_problem9.csv')
df_55

,line,month,total
0,1,1,1310
1,1,2,1300
2,1,3,1305
3,1,4,1200
4,1,5,1301
...,...,...,...
67,3,8,1365
68,3,9,1374
69,3,10,1337
70,3,11,1332


1. 상호작용 효과검정
귀무가설H0 :  지하철 호선과 월별, 승객 수 간 상관관계가 없다.
대립가설H1 :  지하철 호선과 월별, 승객 수 간 상관관계가 있다.

2. 주효과 검정
    2.1) 귀무가설H0 : 지하철 호선별 승객 수 차이가 없다.
         대립가설H1 : 지하철 호선별 승객 수 차이가 있다.
    2.2) 귀무가설H0 : 지하철 월별 승객 수의 차이가 없다.
         대립가설H1 : 지하철 월별 승객 수의 차이가 있다.

In [187]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

model = ols('total ~ C(line)*C(month)', data=df_55).fit()
anova_table = anova_lm(model, typ=3)

anova_table

,sum_sq,df,F,PR(>F)
Intercept,3466344.5000000018626451492309570312500000000000,1.0000000000000000000000000000000000000000,33455.3356568364761187694966793060302734375000,0.0000000000000000000000000000000000000000
C(line),133662.9999999998253770172595977783203125000000,2.0000000000000000000000000000000000000000,645.0225201072377103628241457045078277587891,0.0000000000000000000000000000641794594579
C(month),39424.8333333334594499319791793823242187500000,11.0000000000000000000000000000000000000000,34.5916158908117097325884969905018806457520,0.0000000000000007896178813599910602543906
C(line):C(month),96969.0833333340560784563422203063964843750000,22.0000000000000000000000000000000000000000,42.5406653668050935834799020085483789443970,0.0000000000000000001582231693043414203384
Residual,3730.0000000000000000000000000000000000000000,36.0000000000000000000000000000000000000000,NaN,NaN
